In [32]:
import pandas as pd
import folium
df = pd.read_csv("synth-lieux.csv", encoding='utf-8', sep=';')

coords=df["Adresse"].tolist()
places=df["Place"].tolist()
cleaned_coords = [(lat, lon) for coord in coords if isinstance(coord, str) and ',' in coord for lat, lon in [coord.split(',')]]

coords_with_places = [(coord[0], coord[1], place) for coord, place in zip(cleaned_coords, places)]

print(cleaned_coords)
print(coords_with_places)

# Create a map centered at the first coordinate
map_center = (float(coords_with_places[0][0]), float(coords_with_places[0][1]))
my_map = folium.Map(location=map_center, zoom_start=13)

# Add markers for each coordinate
for loc in coords_with_places:
    lat, lon, place = loc
    folium.Marker(
        location=(float(lat), float(lon)),
        tooltip=f'Place: {place}'
    ).add_to(my_map)

# Save the map to an HTML file
my_map.save("map.html")


[('52.5107745', '13.3896707'), ('52.5025988', '13.4199707'), ('52.4916585', '13.3660462'), ('52.4919769', '13.3646303'), ('52.4919769', '13.3646303'), ('52.4919769', '13.3646303'), ('52.4967592', '13.3618488'), ('52.4932746887207', '13.36770248413086'), ('52.4965733', '13.3653082'), ('52.4977986', '13.3513834'), ('52.4980181', '13.3483763'), ('52.5042512', '13.3582193'), ('52.5021173', '13.3528839'), ('52.49085998535156', '13.369101524353027'), ('52.5104137', '13.376765'), ('52.5138731', '13.3787346'), ('52.5271586', '13.4321467'), ('52.5429726', '13.4264227')]
[('52.5107745', '13.3896707', 'Centre Marc Bloch'), ('52.5025988', '13.4199707', 'Chez Michel'), ('52.4916585', '13.3660462', 'Yorkstrasse'), ('52.4919769', '13.3646303', 'Les Migras'), ('52.4919769', '13.3646303', 'Lesbenberatung'), ('52.4919769', '13.3646303', 'Siegesaüle'), ('52.4967592', '13.3618488', 'Begine'), ('52.4932746887207', '13.36770248413086', 'Blocksberg'), ('52.4965733', '13.3653082', 'Kaiserhof'), ('52.4977986',